In [2]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), "..") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
#---

"C:\\Users\\zacfa\\Documents\\CS\\238\\project\\cs238\\Project.toml"

In [3]:
# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using POMDPModels
using POMDPSimulators
using QMDP

┌ Info: Loading Cairo backend into Compose.jl
└ @ Compose C:\Users\zacfa\.julia\packages\Compose\BYWXX\src\Compose.jl:161
┌ Warning: Package Compose does not have Cairo in its dependencies:
│ - If you have Compose checked out for development and have
│   added Cairo as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Compose
│ Loading Cairo into Compose from project dependency, future warnings for Compose are suppressed.
└ @ nothing nothing:837


## Define sensor and construct POMDP
In the following cell, we first instantiate a sensor. There are two sensors implemented: a lidar sensor and a bump sensor. The lidar sensor receives a single noisy measurement of the distance to the nearest wall in the direction the Roomba is facing. The bump sensor indicates when contact has been made between any part of the Roomba and any wall.

Next, we instantiate the MDP, which defines the underlying simulation environment, assuming full observability. The MDP takes many arguments to specify details of the problem. Feel free to examine the underlying source code (```src/roomba_env.jl```) to gain an understanding for these arguments. One argument we must specify here is the ```config```. This argument, which can take values 1,2, or 3, specifies the room configuration, with each configuration corresponding to a different location for the goal and stairs.

Finally, we instantiate the POMDP. The POMDP takes as an argument the underlying MDP as well as the sensor, which it uses to define the observation model. 

In [4]:
sensor = Bumper() # or Bumper() for the bumper version of the environment
config = 3 # 1,2, or 3
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config));

### Setting up a Particle Filter

To solve a POMDP, we must specify a method for representing the belief state and updating it given an observation and an action. Here, we demonstrate how to instantiate a particle filter, which has been implemented in the package ```ParticleFilters.jl```.

First, we instantiate a resampler, which is responsible for updating the belief state given an observation. We have implemented a resampler for each of the lidar and bumper environments, the source code for which can be found in ```src/filtering.jl```. The first argument for both resamplers is the number of particles that represent the belief state. The lidar resampler takes a low-variance resampler as an additional argument, implemented for us in ```ParticleFilters.jl```, which is responsible for efficiently resampling a weighted set of particles. 

Next, we instantiate a ```SimpleParticleFilter```, which enables us to perform our belief updates, and is implemented for us in ```ParticleFilters.jl```, .

Finally, we pass this particle filter into a custom struct called a ```RoombaParticleFilter```, which takes two additional arguments. These arguments specify the noise in the velocity and turn-rate, used when propegating particles according to the action taken. These can be tuned depending on the type of sensor used.

In [5]:
num_particles = 2000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

### Define a policy

Here we demonstrate how to define a naive policy that attempts navigate the Roomba to the goal. The heuristic policy we define here first spins around for 25 time-steps in order to perform localization, then follows a simple proprtional control law that navigates the robot in the direction of the goal state (note that this policy fails if there is a wall in the way).

First we create a struct that subtypes the Policy abstract type, defined in the package ```POMDPPolicies.jl```. Here, we can also define certain parameters, such as a variable tracking the current time-step.

Next, we define a function that can take in our policy and the belief state and return the desired action. We do this by defining a new ```POMDPs.action``` function that will work with our policy. 

In [6]:
# Define the policy to test
mutable struct ToEnd <: Policy
    ts::Int64 # to track the current time-step.
end

# extract goal for heuristic controller
goal_xy = get_goal_xy(m)
print(goal_xy)

# define a new function that takes in the policy struct and current belief,
# and returns the desired action
function POMDPs.action(p::ToEnd, b::ParticleCollection{RoombaState})
    p.ts += 1
    
#     # turn right 20% of the time
    if p.ts % 20 >= 12 && p.ts % 20 < 16
        return RoombaAct(5.0, 5*pi)
    end
    
    # compute mean belief of a subset of particles
    s = mean(b)
    #print(m.sensor == true)
    #if AA228FinalProject.wall_contact(m,s)
    #    return RoombaAct(3.0,pi)
    #end
    # compute the difference between our current heading and one that would
    # point to the goal
    goal_x, goal_y = goal_xy
    x,y,th = s[1:3]
    ang_to_goal = atan(goal_y - y, goal_x - x)
    #print(string("ang_to_goal: ", string(ang_to_goal)))
    del_angle = wrap_to_pi(ang_to_goal - th)
    #print(string("del_angle: ", string(del_angle)))
    
    # apply proportional control to compute the turn-rate
    Kprop = 2.0
    om = Kprop * del_angle
    
    # always travel at some fixed velocity
    v = 5.0
    
    return RoombaAct(v, om)
end

[15.0, 0.0]

### Simulation and rendering

Here, we will demonstrate how to seed the environment, run a simulation, and render the simulation. To render the simulation, we use the ```Gtk``` package. 

The simulation is carried out using the ```stepthrough``` function defined in the package ```POMDPSimulators.jl```. During a simulation, a window will open that renders the scene. It may be hidden behind other windows on your desktop.

In [ ]:
# first seed the environment
Random.seed!(2)

# reset the policy
p = ToEnd(0) # here, the argument sets the time-steps elapsed to 0
# for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=100))
#     print("hi")
# end
# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, p, belief_updater, max_steps=100))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end

### Evaluation 

Here, we demonstate a simple evaluation of the policy's performance for a few random seeds. This is meant to serve only as an example, and we encourage you to develop your own evaluation metrics.

We intialize the robot using five different random seeds, and simulate its performance for 100 time-steps. We then sum the rewards experienced during its interaction with the environment and track this total reward for the five trials.
Finally, we report the mean and standard error for the total reward. The standard error is the standard deviation of a sample set divided by the square root of the number of samples, and represents the uncertainty in the estimate of the mean value.

In [7]:
using Statistics

total_rewards = []
num_success = 0
num_seeds = 100
skip = 0

for exp = 1:num_seeds
    try
        println(string(exp))

        Random.seed!(exp+skip)
        #srand(exp)

        p = ToEnd(0)
        traj_rewards = 0
        for step in stepthrough(m,p,belief_updater, max_steps=100)
            traj_rewards += step.r
            if step.r > 5
                println("reached goal")
                num_success += 1
                push!(total_rewards, traj_rewards)
                break
            end
        end
    catch ex
        skip += 1
        exp -= 1
        continue
    end
#     traj_rewards = sum([step.r for step in stepthrough(m,p,belief_updater, max_steps=300)])
    
#     push!(total_rewards, traj_rewards)
end

success_rate = (num_success*1.0)/num_seeds
mtr = mean(total_rewards)
score = success_rate*success_rate*mtr
@printf("Percent that reached goal: %.3f%%", success_rate*100)
println()
@printf("Mean Total Reward: %.3f", mtr)
println()
x = 
@printf("Score: %.3f", score)

1
reached goal
2
3
4
5
6
7
8
9
10
reached goal
11
12
13
14
15
16
17
18
19
reached goal
20
21
22
23
24
reached goal
25
26
reached goal
27
reached goal
28
29
reached goal
30
reached goal
31
reached goal
32
33
34
35
36
37
reached goal
38
39
40
41
42
43
44
45
46
reached goal
47
48
49
50
reached goal
51
52
53
54
55
56
reached goal
57
reached goal
58
59
60
61
reached goal
62
reached goal
63
64
65
66
67
reached goal
68
reached goal
69
70
reached goal
71
reached goal
72
73
74
reached goal
75
76
77
78
reached goal
79
80
reached goal
81
82
83
84
85
reached goal
86
reached goal
87
88
89
90
reached goal
91
92
93
reached goal
94
95
96
97
98
99
100
Percent that reached goal: 27.000%
Mean Total Reward: 3.807
Score: 0.278

Experimental stuff

In [ ]:
vlist = [0,1.0,3.0,5.0]
omlist = [-2.0,-1.0,0,1.0,2.0]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist))

num_x_pts = 50
num_y_pts = 50
num_th_pts = 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts)

In [ ]:
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, aspace=aspace, sspace=sspace))

In [ ]:
num_particles = 2000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

In [ ]:
# initialize a solver and compute a policy
solver = QMDPSolver() # from QMDP
policy = solve(solver, m)
belief_updater = updater(policy) # the default QMDP belief updater (discrete Bayesian filter)

# run a short simulation with the QMDP policy
history = simulate(HistoryRecorder(max_steps=10), m, policy, belief_updater)

# look at what happened
for (s, b, a, o) in eachstep(history, "sbao")
    println("State was $s,")
    println("belief was $b,")
    println("action $a was taken,")
    println("and observation $o was received.\n")
end
println("Discounted reward was $(discounted_reward(history)).")